# Re Check Arnoldi
I am going to make sure I understand the output from Arnoldi.  I am going to use our stripey PDE matrix with a few extra non-zeros.

In [30]:
using LinearAlgebra, KrylovKit, SparseArrays
n=1200
A=spzeros(n,n);
for i in 2:n
    A[i,i]=-2.0; A[i,i-1]=1.0; A[i-1,i]=1.0
end
A[1,n]=A[n,1]=1.0;A[1,1]=-2.0
A[4,5]=A[5,4]=-5.0
v=rand(n)

k=44;
iterator = ArnoldiIterator(A, v)
factorization = initialize(iterator)
for i in 1:k-1
@time expand!(iterator, factorization)
end


(VInt, H, res, normRes, b) = factorization

V=zeros(n,k); Idk = Matrix(1.0I,k,k)
v=zeros(n)
for i in 1:k
    V[:,i]=KrylovKit.unproject!(v,VInt,Idk[:,i])
end

  0.000027 seconds (3 allocations: 9.594 KiB)
  0.000012 seconds (2 allocations: 9.594 KiB)
  0.000016 seconds (2 allocations: 9.672 KiB)
  0.000011 seconds (2 allocations: 9.578 KiB)
  0.000013 seconds (2 allocations: 9.828 KiB)
  0.000020 seconds (1 allocation: 9.500 KiB)
  0.000016 seconds (2 allocations: 10.156 KiB)
  0.000024 seconds (2 allocations: 9.641 KiB)
  0.000019 seconds (1 allocation: 9.500 KiB)
  0.000040 seconds (1 allocation: 9.500 KiB)
  0.000037 seconds (2 allocations: 10.828 KiB)
  0.000021 seconds (1 allocation: 9.500 KiB)
  0.000018 seconds (1 allocation: 9.500 KiB)
  0.000024 seconds (1 allocation: 9.500 KiB)
  0.000033 seconds (1 allocation: 9.500 KiB)
  0.000018 seconds (3 allocations: 12.328 KiB)
  0.000018 seconds (1 allocation: 9.500 KiB)
  0.000031 seconds (1 allocation: 9.500 KiB)
  0.000019 seconds (1 allocation: 9.500 KiB)
  0.000062 seconds (1 allocation: 9.500 KiB)
  0.000034 seconds (1 allocation: 9.500 KiB)
  0.000503 seconds (1 allocation: 9.500 KiB

# Checking the output!

The "Hessenberg Matrix" H and the "Orthonormal Basis" V are stored in some funky format!  

In [7]:
H

4×4 KrylovKit.PackedHessenberg{Float64, Vector{Float64}}:
 -0.649145   1.39562   7.89299e-17  -7.15573e-18
  1.39562   -3.26867   1.98797      -1.73472e-17
   ⋅         1.98797  -2.97307       4.30386
   ⋅          ⋅        4.30386      -0.717615

In [9]:
VInt

KrylovKit.OrthonormalBasis{Vector{Float64}}([[0.08983825711983799, 0.09141342403883565, 0.007134438684786458, 0.02871004730575601, 0.11333120590567722, 0.11008366581493809, 0.006622536643451796, 0.0393622416214493, 0.10966427845770524, 0.04933308338987032  …  0.04851621073542553, 0.060771851553656664, 0.10434960498150948, 0.010112291936337647, 0.08171321606257434, 0.0743635684323047, 0.01232231138860188, 0.028619255733110544, 0.1537078763063814, 0.02182206313922496], [-0.005820380154682161, -0.018997803647114304, 0.07916642496072263, -0.42870317014334336, -0.13367626735391464, -0.02060265206236209, 0.10067230927596478, 0.045223146695498845, -0.04259430353499065, 0.07814874688092324  …  0.053922988760452595, 0.05071012601571133, -0.05021232385267749, 0.1235315872348259, -0.01856305395022511, -0.0045993361204265636, 0.0618631517562634, 0.09126419567842874, -0.1126354805999481, 0.15338587687784608], [0.0008170694430253, -0.03940392501060965, -0.17969201253091172, 0.08229434990684091, 0.90

You can convert the H to a normal format using Matrix.

In [5]:
Matrix(H)

4×4 Matrix{Float64}:
 -0.649145   1.39562   7.89299e-17  -7.15573e-18
  1.39562   -3.26867   1.98797      -1.73472e-17
  0.0        1.98797  -2.97307       4.30386
  0.0        0.0       4.30386      -0.717615

You can not convert the basis to a matrix using matrix.  You need to use the trick up above.

In [12]:
Matrix(VInt)

LoadError: MethodError: no method matching (Matrix{T} where T)(::KrylovKit.OrthonormalBasis{Vector{Float64}})
[0mClosest candidates are:
[0m  (Matrix{T} where T)([91m::Union{QR, LinearAlgebra.QRCompactWY}[39m) at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\LinearAlgebra\src\qr.jl:434
[0m  (Matrix{T} where T)([91m::QRPivoted[39m) at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\LinearAlgebra\src\qr.jl:441
[0m  (Matrix{T} where T)([91m::SuiteSparse.CHOLMOD.Dense{T}[39m) where T at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\SuiteSparse\src\cholmod.jl:990
[0m  ...

The question last week was: How in the heck are they stored and why are they stored that way?  Lets see if we can work out the how for H using the  documentation
https://jutho.github.io/KrylovKit.jl/latest/man/implementation/

In [18]:
H

4×4 KrylovKit.PackedHessenberg{Float64, Vector{Float64}}:
 -0.649145   1.39562   7.89299e-17  -7.15573e-18
  1.39562   -3.26867   1.98797      -1.73472e-17
   ⋅         1.98797  -2.97307       4.30386
   ⋅          ⋅        4.30386      -0.717615

In [26]:
H.data

14-element Vector{Float64}:
 -0.6491445535952282
  1.395615326131287
  1.3956153261312874
 -3.2686655553495525
  1.9879692160106683
  7.892991815694472e-17
  1.9879692160106683
 -2.9730674767799607
  4.3038642795977
 -7.15573433840433e-18
 -1.734723475976807e-17
  4.303864279597698
 -0.7176145213985711
  1.6777589676245104

# Lanczos Version
Lets see if the Lanczos version is lighter weight. 

In [32]:
k=4;
iterator = LanczosIterator(A, v)
factorization = initialize(iterator)
for i in 1:k-1
@time expand!(iterator, factorization)
end

(VInt, T, res, normRes, b) = factorization

V=zeros(n,k); Idk = Matrix(1.0I,k,k)
v=zeros(n)
for i in 1:k
    V[:,i]=KrylovKit.unproject!(v,VInt,Idk[:,i])
end

  0.000032 seconds (4 allocations: 9.641 KiB)
  0.000015 seconds (1 allocation: 9.500 KiB)
  0.000010 seconds (1 allocation: 9.500 KiB)


In [33]:
T

4×4 SymTridiagonal{Float64, Vector{Float64}}:
 -1.9862    1.41901    ⋅         ⋅ 
  1.41901  -1.99878   1.00525    ⋅ 
   ⋅        1.00525  -2.00537   1.21279
   ⋅         ⋅        1.21279  -2.00907

In [42]:
T[1:12]

12-element SparseVector{Float64, Int64} with 8 stored entries:
  [1 ]  =  -1.9862
  [2 ]  =  1.41901
  [5 ]  =  1.41901
  [6 ]  =  -1.99878
  [7 ]  =  1.00525
  [10]  =  1.00525
  [11]  =  -2.00537
  [12]  =  1.21279

In [46]:
fieldnames(SymTridiagonal)

(:dv, :ev)

In [50]:
fieldtypes(SymTridiagonal)

(AbstractVector{T} where T, AbstractVector{T} where T)

Hit tab to see what will auto complete! 

In [49]:
T.

4-element Vector{Float64}:
 1.4190109646396867
 1.0052529354598954
 1.2127885589649672
 0.7215560812472576